In [ ]:
def invisible(ax):
    # ax=plt.gca()  #gca:get current axis得到当前轴
    #设置图片的右边框和上边框为不显示
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['bottom'].set_color('none')
    ax.axes.get_yaxis().set_visible(False)
    # x 轴不可见
    ax.axes.get_xaxis().set_visible(False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.manifold import TSNE
data = pickle.load(open('ana.pkl','rb'))
data.keys()

In [ ]:
from tqdm import tqdm
import time


# print(os.getcwd())


def get_hour(time_str, filename):
    hour = None
    try:
        msg_time = int(time_str)
        hour = time.strftime("%H", time.localtime(msg_time))
        hour = int(hour)
    except:
        if '170w' in filename:  # fixed in 11.15, however, in this way, more datasets will be removed
            ts = time.strptime(time_str, '%Y-%m-%d-%H:%M:%S')
            hour = ts.tm_hour
        elif 'castle' in filename:
            # for data castle weibo
            hour = int(time_str[:2])
        elif 'smp' in filename:
            ts = time.strptime(time_str, '%Y-%m-%dT%H:%M:%S')
            hour = ts.tm_hour
        else:
            print(time_str)
            print('wrong time format')
    return hour
import networkx as nx
def construct_G(filename='/home/ta/yh/open-cas/rawdata/dataset_weibo.txt'):
    G = nx.DiGraph()
    cascade_count={}
    # filename='./dataset_citation.txt'
    discard_midnight = 0
    discard_outer = 0       
    file = open(filename)
    cascades_total = dict()
    year_grow = dict()
    num_lines = sum(1 for line in open(filename, 'r'))
    for line in tqdm(file, total=num_lines):
        parts = line.strip().split("\t")
        if len(parts) != 5:
            print('wrong format!')
            continue
        cascadeID = parts[0]
        # print cascadeID
        n_nodes = int(parts[3])

        path = parts[4].split(" ")
        if n_nodes != len(path):
            print(cascadeID,' wrong number of nodes', n_nodes, len(path),path)
            exit(0)
        # for p in path:
        #     nodes = p.split(":")[0].split("/")
        #     time_ = int(int(p.split(":")[1]))
        #     if time_<=3600:
        #         num+=1
        # if num>100 :
        #     discard_outer+=1
        #     continue
        # if n_nodes<5 or n_nodes>100:
        #     continue
        hour = get_hour(parts[2], filename)
        # to keep the same with
        if hour <= 7 or hour >= 19:  # 8-18
            discard_midnight += 1
            continue
        observation_path = []
        # print(cascadeID)
        for p in path:
            nodes = p.split(":")[0].split("/")
            time_ = int(int(p.split(":")[1])/360)
            try:
                year_grow[time_]+=1
            except:
                year_grow[time_]=1
            nodes_ok = True
            for n in nodes:
                if int(n) == -1:  # delete invalid id
                    nodes_ok = False
            if not (nodes_ok):
                print("error id at cas_id {}".format(cascadeID))
                print(nodes)
                continue
            # print nodes
            time_now = int(p.split(":")[1])

            observation_path.append(",".join(nodes) + ":" + str(time_now))
            for i in range(1, len(nodes)):
                G.add_edge(nodes[i - 1] , nodes[i])
        cascades_total[cascadeID] = hour

    n_total = len(cascades_total)
    print("total_readin:", num_lines)
    print("discard_midnight:", discard_midnight)
    print("discard_outer:", discard_outer)
    print('total:', n_total)
    # import operator
    # sorted_msg_time = sorted(cascades_total.items(),
    #                          key=operator.itemgetter(1))
    # 计算每个节点的 PR 值，并作为节点的 pagerank 属性
    print('computing pange rank')
    pagerank = nx.pagerank(G)
    # 将 pagerank 数值作为节点的属性
    nx.set_node_attributes(G, name = 'pagerank', values=pagerank)
    print('computing done')

    return G
    # 划分数据集
G = construct_G('/home/ta/yh/open-cas/rawdata/dataset_citation.txt')
page_rank = nx.get_node_attributes(G, "pagerank")
# page_rank['1130594']

In [7]:
gnn_o=data['gnn_o']
X = []
y = []
nodeid = data['id']
for index,batch_id in enumerate(nodeid):
    step=len(batch_id)-1
    emb = gnn_o[index][step][0]
    for i,node_id in enumerate(batch_id):
        X.append(emb[i])
        y.append(page_rank[str(node_id)])


In [28]:
avg = np.average(y)
top = sorted(y,reverse=True)[int(len(y)*0.01)]
sample = int(len(y)*0.01)
print(sample)
print(avg)
label = []
new_X = []
for i,x in zip(y,X):
    if i>top:
        label.append(2)
        new_X.append(x)
    # elif i > avg*0.9:
    #     label.append(1)
    elif sample > 0:
        sample-=1
        label.append(0)
        new_X.append(x)
# X label
label=np.array(label)
np.sum(label==0),np.sum(label==1),np.sum(label==2)

1873
3.2269627339524915e-06


(1873, 0, 1872)

In [31]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_X, label,test_size=0.25, stratify = label)
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
clf.predict_proba(X_test)
clf.predict(X_test)
mlp_predictions = clf.predict(X_test)
cm = confusion_matrix(y_test, mlp_predictions)
print(clf.score(X_test, y_test),cm)



0.5368196371398078 [[ 48 421]
 [ 13 455]]


In [ ]:
Ytest = np.array(data['Ytest'])
Xtest = np.array(data['Xtest'])
time = data['time']
# Ytest = np.power(2,Ytest)
# Hs = data['hidden_states']
Hs = np.array(data['hidden_states']).reshape(-1,100,32)
len(Ytest),Hs.shape
# print(len(Hs[0][0][0]), len(Xtest[0]),len(time))

In [ ]:
Hs.shape,Xtest.shape,Ytest.shape

In [ ]:
h_size,g_size = [],[]
for h in Hs:
    size = 1
    for step_h in h:
        if step_h.all()!=0:
            h_size.append(step_h)
            g_size.append(size)
            size+=1
h_size = np.array(h_size)
g_size = np.array(g_size)
h_size.shape,g_size.shape

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
plt.close()
invisible(plt.gca())
dHs,dIs = shuffle(h_size,g_size,random_state=42)
X = TSNE(n_components=2,random_state=42,learning_rate=25,n_iter=6000,verbose=1).fit_transform(dHs[:80000])
sc = plt.scatter(X[:, 0], X[:, 1],s=1, c=dIs[:80000], cmap='viridis_r')
plt.colorbar(sc)
plt.savefig('output-gz.png',dpi=600)

In [ ]:
sum_pool_o = np.sum(Hs,axis=1)
sum_pool_o.shape

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
plt.close()
invisible(plt.gca())

dHs,dIs = shuffle(sum_pool_o,Ytest,random_state=42)
X = TSNE(n_components=2,random_state=42,verbose=1).fit_transform(dHs[:10000])
sc = plt.scatter(X[:, 0], X[:, 1],s=1, c=dIs[:10000], cmap='viridis_r')
plt.colorbar(sc)
plt.savefig('output-future_growth.png',dpi=600)

In [ ]:
ob_time = 3600
interval = 180
hiddens  = []
graph_size = []
for hs,ts in zip(Hs,time):
    for time_interval in range(180,3600+1,180):
        size = 0
        for idx,t in enumerate(ts):
            
            if t<time_interval :
                size+=1
            elif t>=time_interval:
                hiddens.append(hs[idx-1])
                graph_size.append(size)
                break
            if idx==len(ts)-1:
                hiddens.append(hs[idx])
                graph_size.append(size)
                break

            
print(np.array(hiddens).shape)




In [ ]:
hiddens = np.array(hiddens)
graph_size = np.array(graph_size)
print(hiddens.shape,graph_size.shape)

In [ ]:
hidden_graph_size = []
graph_size_size = []
for gz,rnn_hidden in zip(graph_size.reshape(-1,20),hiddens.reshape(-1,20,32)):
    hidden_graph_size.append(rnn_hidden[0])
    graph_size_size.append(gz[0])
    for i in range(len(gz[1:])):
        if gz[i]==gz[i+1]:
            continue
        hidden_graph_size.append(rnn_hidden[i+1])
        graph_size_size.append(gz[i+1])




In [ ]:
from sklearn.utils import shuffle

diff_inc,diff_hidden,hist_hidden = [],[],[]
for inc,rnn_hidden in zip(graph_size.reshape(-1,20),hiddens.reshape(-1,20,32)):
    print(inc)
    diff_inc.append(inc[1:]-inc[:-1])
    hist_hidden.append(rnn_hidden[:-1])
    diff_hidden.append(rnn_hidden[1:]-rnn_hidden[:-1])
diff_inc=np.array(diff_inc).reshape(-1)
hist_hidden=np.array(hist_hidden).reshape(-1,32)
diff_hidden=np.array(diff_hidden).reshape(-1,32)
print(diff_inc.shape,hist_hidden.shape,diff_hidden.shape)
# diff_hidden,diff_inc,hist_hidden=shuffle(diff_hidden,diff_inc,hist_hidden)
print(diff_inc[:100])

In [ ]:
np.max(diff_inc),np.min(diff_inc)

In [ ]:
dHs,dIs = [],[]
az = 0
for h,i in zip(diff_hidden,diff_inc):
    if i <=1:
        continue
    if h.all()==0:
        az+=1
        continue
    dHs.append(h)
    dIs.append(np.log2(i))
dHs = np.array(dHs)
dIs = np.array(dIs)
print(len(dHs))
print(az)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
plt.close()
print(len(dHs))
invisible(plt.gca())

dHs,dIs = shuffle(dHs,dIs,random_state=42)
X = TSNE(n_components=2,random_state=42,verbose=1).fit_transform(dHs[:10000])
sc = plt.scatter(X[:, 0], X[:, 1],s=1, c=dIs[:10000], cmap='viridis_r')
plt.colorbar(sc)
plt.savefig('diff.png',dpi=600)

In [ ]:
diff_inc.min(),diff_inc.max()